<a href="https://colab.research.google.com/github/mohammad2682/FVS-SVM-RKHS/blob/main/SVM_RKHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import svm

In [ ]:
!git clone https://github.com/mohammad2682/FVS-SVM-RKHS

Cloning into 'FVS-SVM-RKHS'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 74 (delta 27), reused 41 (delta 17), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [ ]:
def sigma(df):
  N = df.shape[0]
  sigma = []
  for i in range(N):
    for j in range(i+1, N):
      x_i = df.iloc[[i], :-1]
      x_j = df.iloc[[j], :-1]
      sigma.append(np.linalg.norm(x_i.values - x_j.values))
  return 0.1 * np.max(sigma)

In [ ]:
def K(FV1, FV2, sigma):
  dist = np.linalg.norm(FV1-FV2)
  return np.exp(-1*np.square(dist)/(2*sigma))

In [ ]:
def give_KSS(S, sigma):
  KSS = []
  for vec1 in S:
    for vec2 in S:
      k = K(np.array(vec1), np.array(vec2), sigma = sigma)
      KSS.append(k)

  return np.array(KSS).reshape((len(S), len(S)))

In [ ]:
def give_KSx(S, x, sigma):
  KSx = []
  for s in S:
    kx = K(np.array(s), x, sigma = sigma)
    KSx.append(kx)
  return np.array(KSx).reshape((len(S), 1))

In [ ]:
def minor_major(df):
  cls = df.iloc[:,-1].unique()
  cls0 = df[df.iloc[:,-1]==cls[0]].shape[0]
  cls1 = df[df.iloc[:,-1]==cls[1]].shape[0]
  minor = cls[0]
  major = cls[1]
  if cls0>cls1:
    minor = cls[1]
    major = cls[0]
  return minor, major

In [ ]:
def give_landa(df, minor_cls, major_cls):
  minor = df[df["Class"] == minor_cls]
  major = df[df["Class"] == major_cls]
  landa = []
  for col in df.columns[:-1]:
    mean_minor = minor[col].mean()
    mean_major = major[col].mean()
    var_minor = minor[col].var()
    var_major = major[col].var()
    landa_i = np.square(mean_minor - mean_major)/(var_minor - var_major)
    landa.append(landa_i)
  return landa

In [ ]:
df = pd.read_csv("FVS-SVM-RKHS/datasets/pima.csv")
df.head()

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,14,175,62,30,0,33.6,0.212,38,tested_positive
1,4,146,78,0,0,38.5,0.520,67,tested_positive
2,15,136,70,32,110,37.1,0.153,43,tested_positive
3,3,107,62,13,48,22.9,0.678,23,tested_positive
4,3,169,74,19,125,29.9,0.268,31,tested_positive


In [ ]:
df_test1 = df.iloc[:3, :]
df_test2 = df.iloc[-7:, :]
df_test = pd.concat([df_test1, df_test2]).reset_index(drop=True)
df_test = df_test[["preg", "plas", "class"]]

In [ ]:
df_test

,preg,plas,class
0,14,175,tested_positive
1,4,146,tested_positive
2,15,136,tested_positive
3,6,114,tested_negative
4,6,92,tested_negative
5,5,117,tested_negative
6,4,83,tested_negative
7,7,119,tested_negative
8,1,95,tested_negative
9,1,181,tested_positive


In [ ]:
sig = sigma(df_test)
minor_cls, major_cls = minor_major(df_test)

In [ ]:
FV1 = df_test.iloc[0, :-1].values
tau = 0.1 #minimum of 0.001, 1/N
S = [list(FV1)]
cnt = 1
for vec in df_test.iloc[1:, :-1].values:
  KSx = give_KSx(S, vec, sigma = sig)
  KSS = give_KSS(S, sigma = sig)
  LF = abs(1-(np.dot(np.dot(np.transpose(KSx), np.linalg.inv(KSS)), KSx)))
  #print(LF)
  if LF > tau:
    S.append(list(vec))
    cnt += 1
    if cnt == 5:
      break

cls = []
cnt = 0
#sort Data
for vector in df_test.iloc[1:, :].values:
  vec = np.array(list(vector[:-1]))
  cls.append(vector[-1])
  KSx = give_KSx(S, vec, sigma = sig)
  KSS = give_KSS(S, sigma = sig)
  DSS = np.sqrt(KSS)
  beta = np.dot(np.dot(np.transpose(KSx), np.linalg.inv(KSS)), DSS)
  if cnt == 0:
    B = beta
    cnt = -1
    continue
  B = np.concatenate((B, beta))

B_df = pd.DataFrame(B)
B_df["Class"] = cls

landa = give_landa(B_df, minor_cls, major_cls)

landa_t = np.sum(landa)/len(S)

In [ ]:
landa_t

0.03830932513633186